# <center>On Portability and Performance Versatility in Nonlinear Solid and Fluid Mechanics Using libCEED and PETSc</center>

<br>
<br>
<br>

### <center>Leila Ghaffari<sup>1</sup>, 
<br>
<center>Jeremy L. Thompson<sup>1</sup>, Valeria Barra<sup>1,2</sup>, Rezgar Shakeri<sup>3</sup>, Karen Stengel<sup>1</sup>, and Jed Brown<sup>1</sup></center>  

<br>
  
<center><sub><sup><sup>1</sup> Department of Computer Science, CU Boulder</sup></sub></center>  
<center><sub><sup><sup>2</sup> Department of Environmental Science and Engineering, California Institute of Technology</sup></sub></center>
<center><sub><sup><sup>3</sup> Department of Civil Engineering, CU Boulder</sup></sub></center>  

<br>
<br>
<br>

<center>SIAM PP22, 23 Feb 2022</center>

<img align="left" src="ecp.png" width="500"/>  <img align="right" src="ceed-exascale-landscape.png" width="600"/>

# To assemble or not to assemble?

<img align="left" src="assemble-or-not.png" width="80%"/>

# [libCEED](https://libceed.readthedocs.io): Efficient Extensible Discretization  

<img align="left" src="libceed-badges.png" width="100%"/>

* High-order finite/spectral element (FEM/SEM) library exploiting tensor-product structure

* Open source (BSD-2 license) C library with Fortran, Python, Julia, and Rust interfaces

* [libCEED User Manual](https://libceed.readthedocs.io/en/latest/)



<img align="left" src="libceed-qr-code.png" width="200"/>   <img align="left" src="libceed-docs-qr-code.png" width="200"/> 

# libCEED's mission


* Purely algebraic FEM library

* Single source vanilla C for QFunctions

* Backend plugins with run-time selection
    * e.g., `./bps -ceed /gpu/cuda`

* Same source code can call multiple CEEDs with different backends

* Available via MFEM, PETSc, Nek5000




<img align="right" src="libCEEDBackends.png" width="100%"/>

<center><img align="center" src="libCEEDDecomposition.png" width="2000"/></center>

* <font color='red'>$P$</font> - Process decomposition 
* $\mathcal E$</font> - Element restriction
* <font color='blue'>$B$</font> - Basis (DoFs-to-Qpts) evaluator 
* <font color='green'>$D$</font> - Operator at quadrature point 

# Toward real-world problems


## <center>Solid mechanics mini-app</center>

#### Neo-Hookean Hyperelasticity at Finite Strain
----

### Strong form:

$$
- \nabla_X \cdot \boldsymbol{P} - \rho_0 \boldsymbol{g} = \boldsymbol{0}
$$

where $\boldsymbol{P} = \boldsymbol{F} \, \boldsymbol{S}$

### Weak form:  

$$ 
\int_{\Omega_0}{\nabla_X \boldsymbol{v} \colon \boldsymbol{FS}} \, dV
 - \int_{\Omega_0}{\boldsymbol{v} \cdot \rho_0 \boldsymbol{g}} \, dV
 - \int_{\partial \Omega_0}{\boldsymbol{v} \cdot (\boldsymbol{FS} \cdot \hat{\boldsymbol{N}})} \, dS
 = 0, \quad \forall \boldsymbol v \in \mathcal V,
$$

#### Initial vs Current configuration
-----

\begin{equation} \label{residual}
    {\underbrace{\nabla_X \boldsymbol{v} \colon \boldsymbol{FS}}_{\text{Initial Residual}}}
     \xrightarrow[]{\text{push forward}}
     {\underbrace{\nabla_x \boldsymbol{v} \colon \boldsymbol{\tau}}_{\text{Current Residual}}}
\end{equation}

\begin{equation} \label{Jacobian}
    {\underbrace{\nabla_X\boldsymbol{v}\colon \Big(\text{d}\boldsymbol{F}\boldsymbol{S} + \boldsymbol{F}\text{d}\boldsymbol{S}\Big)}_\text{Initial Jacobian}}
     \xrightarrow[]{\text{push forward}}
     {\underbrace{\nabla_x\boldsymbol{v}\colon \Big(\text{d}\boldsymbol{\tau} -\boldsymbol{\tau}(\nabla_x \text{d}\boldsymbol{u})^T \Big)}_\text{Current Jacobian}}
\end{equation}

<img align="center" src="solids_initial_vs_current.png" width="100%">

# Problem space

<img src="holes-mesh.gif" width="90%" align="left">


## Hex
<img align="left" src="vtu_output/hex_res0.png" width="300"> 
<img align="left" src="vtu_output/hex_res2.png" width="300"> 

## Tet
<img align="left" src="vtu_output/tet_res0.png" width="300"> 
<img align="left" src="vtu_output/tet_res2.png" width="300">

## $p$-Refinement Study

<img align="center" src="p-refinement.png" width="70%">

## libCEED-Enzyme (WIP)  <img align="right" src="ceed-logo-no-words.png" width="100"/> <img align="right" src="enzyme-logo.svg" width="100"/>  

* Constitutive modeling

$$
\boldsymbol S = \lambda (trace \boldsymbol E) \boldsymbol I_3 + 2 \mu \boldsymbol E
$$

* Newton linearization

$$
d \boldsymbol S = \frac{\partial \boldsymbol S}{\partial \boldsymbol E} \!:\! d  \boldsymbol E
= \lambda (\boldsymbol C^{-1} \!:\! d \boldsymbol E) \boldsymbol C^{-1}
  + 2 (\mu - \lambda \log J) \boldsymbol C^{-1} d \boldsymbol E \, \boldsymbol C^{-1}
$$

<sub><sup>$S$ = Second Piola-Kirchhoff tensor
    
<sub><sup>$E$ = Green-Lagrange strain tensor

<sub><sup>$C$ = Cauchy-Green tensor ($\boldsymbol C = \boldsymbol I_3 + 2 \boldsymbol E$ )

<sub><sup>$J = \lvert \boldsymbol F \rvert = \sqrt{\lvert \boldsymbol C \rvert}$

<sub><sup>$\mu$, $\lambda$ = Lame constants

## libCEED-Enzyme (WIP)  <img align="right" src="ceed-logo-no-words.png" width="100"/> <img align="right" src="enzyme-logo.svg" width="100"/>  

```c
// Compute Second Piola-Kirchhoff S(E)
int computeS(CeedScalar S[6], CeedScalar E[6], const CeedScalar lambda, const CeedScalar mu);

// Get tape size
__enzyme_augmentsize(computeSfwd, enzyme_dup, enzyme_dup, enzyme_const, enzyme_const);

// Forward mode
__enzyme_augmentfwd((void *)computeS, enzyme_allocated, sizeof(tape[0]), enzyme_tape, tape, S, 
                    (double *)NULL, E, (double *)NULL, enzyme_const, lambda, enzyme_const, mu);

// Reverse mode
__enzyme_reverse((void *)computeS, enzyme_allocated, sizeof(tape[0]), enzyme_tape, tape, 
                 (double *)NULL, dS, (double *)NULL, dE, enzyme_const, lambda, enzyme_const, mu);

```

# Outlook

  * Robust matrix-free solvers
  * Strong scaling
  * SYCL/DPC++ backend
  * Fluids - Shock-capturing
  * Fluids - Turbulence modeling
  * We invite contributors and friendly users



&nbsp;

&nbsp;

&nbsp;

```
This material is based upon work supported 
by the U.S. Department of Energy, Office of 
Science, Office of Advanced Scientific 
Computing Research under Award Number DE-SC0016140.

```

<img align="center" src="baby-ratel.jpeg" width="60%"/>

<img align="left" src="ratel-gitlab.png" width="100%"/>


[https://gitlab.com/micromorph/ratel](https://gitlab.com/micromorph/ratel)